# Regresión Lineal: Ejemplo 1

Este ejemplo es una modificación del ejemplo de la documentación de Spark.



In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Ejemplo1RegLineal').getOrCreate()

In [3]:
# Cargar los datos de entrenamiento (en formato libsvm: formato que MLlib acepta por defecto)
archivo = "/content/datos_regresion.libsvm.txt"      # Indicar la ruta correcta
datos = spark.read.format("libsvm").load(archivo)    # Leer los datos

In [4]:
# Visualizar los 5 primeros datos sin truncar las columnas
datos.show(5, truncate=False)

# El formato de "features" es: (#atributos, [nombres_atributos], [atrib1, atrib2, ...])

+-------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|label              |features                                                                                                                                                                                                                             |
+-------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|-9.490009878824548 |(10,[0,1,2,3,4,5,6,7,8,9],[0.4551273600657362,0.36644694351969087,-0.38256108933468047,-0.4458430198517267,0.33109790358914726,0.8067445293443565,-0.2624341731773887,-0.44850386111659524,-0.07269284838169332,0.5658035575800

In [5]:
# Mostrar datos en formato de lista
datos.take(5)

[Row(label=-9.490009878824548, features=SparseVector(10, {0: 0.4551, 1: 0.3664, 2: -0.3826, 3: -0.4458, 4: 0.3311, 5: 0.8067, 6: -0.2624, 7: -0.4485, 8: -0.0727, 9: 0.5658})),
 Row(label=0.2577820163584905, features=SparseVector(10, {0: 0.8387, 1: -0.127, 2: 0.4998, 3: -0.2269, 4: -0.6452, 5: 0.1887, 6: -0.5805, 7: 0.6519, 8: -0.6556, 9: 0.1749})),
 Row(label=-4.438869807456516, features=SparseVector(10, {0: 0.5026, 1: 0.1421, 2: 0.16, 3: 0.505, 4: -0.9372, 5: -0.2842, 6: 0.6356, 7: -0.1646, 8: 0.9481, 9: 0.4268})),
 Row(label=-19.782762789614537, features=SparseVector(10, {0: -0.0389, 1: -0.4167, 2: 0.8997, 3: 0.641, 4: 0.2733, 5: -0.2618, 6: -0.2795, 7: -0.1307, 8: -0.0854, 9: -0.0546})),
 Row(label=-7.966593841555266, features=SparseVector(10, {0: -0.062, 1: 0.6546, 2: -0.6979, 3: 0.6677, 4: -0.0794, 5: -0.4389, 6: -0.6081, 7: -0.6415, 8: 0.7314, 9: -0.0268}))]

## 1. Creación del modelo

Los datos ya tienen el formato de dos columnas que Spark espera. Estas columnas son: "label" y "features". 

* La columna "label" necesita ser numérica, sea de valor real (para una regresión numérica), o de valor entero (para clasificación). 
* La columna "feature" contiene un vector con todos los atributos ("features") que corresponden a dicha fila. Usualmente para obtener esta columna se combina varias columnas de atributos en una sola columna

In [6]:
# Modelo de regresión lineal
from pyspark.ml.regression import LinearRegression

# Se especifica las columnas que harán las veces de 
#  * "featuresCol" - columna de atributos (solo 1 columna)
#  * "labelCol" - columna que contiene las etiquetas
#  * "predictionCol" - columna que se creará

# Creación del modelo (se puede pasar parámetros adicionales para, por ejemplo, regularización)
reglin = LinearRegression(featuresCol='features', labelCol='label', predictionCol='prediccion', )

## 2. Entrenamiento y evaluación usando solo datos de entrenamiento

In [7]:
# Entrenar el modelo (ajustar el modelo a los datos)
modeloRL = reglin.fit(datos)

In [8]:
# Mostrar los coeficientes y el intercepto
print("Coeficientes (w1, w2, ...):", modeloRL.coefficients) # Para cada atributo
print("Intercepto (w0):", modeloRL.intercept)

Coeficientes (w1, w2, ...): [0.0073350710225801715,0.8313757584337543,-0.8095307954684084,2.441191686884721,0.5191713795290003,1.1534591903547016,-0.2989124112808717,-0.5128514186201779,-0.619712827067017,0.6956151804322931]
Intercepto (w0): 0.14228558260358093


El atributo "summary" contiene información sobre el modelo entrenado (evaluación sobre el conjunto de entrenamiento)



In [9]:
# Resumir el modelo usando los residuos (valor real - valor predicho)
modeloRL.summary.residuals.show(5)

# Métricas de evaluación
print("RMSE:", modeloRL.summary.rootMeanSquaredError)
print("r2:", modeloRL.summary.r2)

+-------------------+
|          residuals|
+-------------------+
|-11.011130022096554|
| 0.9236590911176538|
|-4.5957401897776675|
|  -20.4201774575836|
|-10.339160314788181|
+-------------------+
only showing top 5 rows

RMSE: 10.16309157133015
r2: 0.027839179518600154


## 3. Entrenamiento y evaluación usando datos de entrenamiento y prueba

Se realizará la separación de los datos en entrenamiento (training) y prueba (test). Usualmente se usa criterio de 70/30, 60/40 u 80/20 (depende de cuántos datos se tenga y cuán bien balanceados estén.

In [10]:
# Separación de datos (70% / 30%)
train_data, test_data = datos.randomSplit([0.7,0.3])

# Mostrar datos de entrenamiento y de prueba
train_data.show(5)
test_data.show(5)

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
|-28.571478869743427|(10,[0,1,2,3,4,5,...|
|-26.805483428483072|(10,[0,1,2,3,4,5,...|
|-26.736207182601724|(10,[0,1,2,3,4,5,...|
|-21.432387764165806|(10,[0,1,2,3,4,5,...|
|-20.212077258958672|(10,[0,1,2,3,4,5,...|
+-------------------+--------------------+
only showing top 5 rows

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
|-28.046018037776633|(10,[0,1,2,3,4,5,...|
| -23.51088409032297|(10,[0,1,2,3,4,5,...|
|-23.487440120936512|(10,[0,1,2,3,4,5,...|
|-22.949825936196074|(10,[0,1,2,3,4,5,...|
|-22.837460416919342|(10,[0,1,2,3,4,5,...|
+-------------------+--------------------+
only showing top 5 rows



Entrenamiento de los datos

In [11]:
# Entrenamiento con datos de entrenamiento (train)
modeloRL = reglin.fit(train_data)

In [12]:
# Evaluación con datos de prueba (test)
resultados = modeloRL.evaluate(test_data)

# Resultados (prediccion: valores estimados con regresión)
resultados.predictions.show(5)

+-------------------+--------------------+--------------------+
|              label|            features|          prediccion|
+-------------------+--------------------+--------------------+
|-28.046018037776633|(10,[0,1,2,3,4,5,...| -0.2166642805332346|
| -23.51088409032297|(10,[0,1,2,3,4,5,...|3.453552196526765...|
|-23.487440120936512|(10,[0,1,2,3,4,5,...| 0.06623610203189884|
|-22.949825936196074|(10,[0,1,2,3,4,5,...|    3.82847458752515|
|-22.837460416919342|(10,[0,1,2,3,4,5,...|  -4.025084613034198|
+-------------------+--------------------+--------------------+
only showing top 5 rows



In [13]:
# Residuos (valores reales - valores estimados)
resultados.residuals.show(5)

# RMSE
print("RMSE:", resultados.rootMeanSquaredError)

+-------------------+
|          residuals|
+-------------------+
|  -27.8293537572434|
|-23.511229445542625|
| -23.55367622296841|
|-26.778300523721224|
|-18.812375803885143|
+-------------------+
only showing top 5 rows

RMSE: 11.116598780247962


### Predicción para datos que solo contienen atributos

Se asumirá que los datos no tienen etiquetas (valores correctos). Con este fin, se escogerá solo la columna "features" usando transform(). 

In [14]:
# Escoger solo los atributos de prueba (descartar las etiquetas)
datos_prueba = test_data.select('features')
datos_prueba.show(5)

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 5 rows



In [15]:
# Realizar predicción sobre los datos de prueba
predicciones = modeloRL.transform(datos_prueba)
predicciones.show(5)

+--------------------+--------------------+
|            features|          prediccion|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...| -0.2166642805332346|
|(10,[0,1,2,3,4,5,...|3.453552196526765...|
|(10,[0,1,2,3,4,5,...| 0.06623610203189884|
|(10,[0,1,2,3,4,5,...|    3.82847458752515|
|(10,[0,1,2,3,4,5,...|  -4.025084613034198|
+--------------------+--------------------+
only showing top 5 rows

